In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
from datetime import datetime
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
import pytz
from pandas.tseries.offsets import Hour
from pandas.tseries.frequencies import to_offset
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
from pmdarima.arima import auto_arima
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error


In [2]:
#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)

In [3]:
autoAR = auto_arima(df['310_PV_1'], trace = True, random_state = 42)
autoAR.summary()

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=166796.330, Time=14.73 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=225269.082, Time=0.47 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=167072.712, Time=1.04 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=195593.186, Time=4.38 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=226331.170, Time=0.26 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=166797.766, Time=6.06 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=166794.623, Time=6.51 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=166807.657, Time=3.13 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=166798.578, Time=1.52 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=166796.403, Time=18.34 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=166795.512, Time=1.98 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=166796.364, Time=18.37 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=166860.797, Time=1.15 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0] intercept
Total fit time: 77.986 

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                35136
Model:               SARIMAX(2, 0, 1)   Log Likelihood              -83392.312
Date:                Mon, 08 May 2023   AIC                         166794.623
Time:                        11:54:07   BIC                         166836.958
Sample:                    01-01-2020   HQIC                        166808.106
                         - 01-01-2021                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0986      0.015      6.581      0.000       0.069       0.128
ar.L1          1.1309      0.025     45.026      0.000       1.082       1.180
ar.L2         -0.2250      0.023     -9.904      0.000      -0.269      -0.180
ma.L1         -0.1531      0.025     -6.094      0.000      -0.202      -0.104
sigma2         6.7456      0.019    353.055      0.000       6.708       6.783
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):            702320.13
Prob(Q):                              0.97   Prob(JB):                         0.00
Heteroskedasticity (H):               0.47   Skew:                            -0.76
Prob(H) (two-sided):                  0.00   Kurtosis:                        24.85
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
tss = TimeSeriesSplit(n_splits = 5, test_size = 20, gap = 0)

In [5]:
mse = []

for train_indx, val_indx in tss.split(df):
    train = df.iloc[train_indx]
    test = df.iloc[val_indx]
    
    mod = ARIMA(train['310_PV_1'], order = (2,0,1))
    result = mod.fit()
    print(result.summary())
    
    start = len(train)
    end = len(train)+1
    
    pred = result.predict(start = start, end = end)
    
    
    error = 0.5*(test['310_PV_1'][0] - pred[0])**2
    mse.append(error)
    
    #print(pred)
    
    #print(test['309_WIND_1'][0])
    #print(pred[0])

print(mse)

    

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35036
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -83189.012
Date:                Mon, 08 May 2023   AIC                         166388.024
Time:                        11:55:08   BIC                         166430.345
Sample:                    01-01-2020   HQIC                        166401.504
                         - 12-30-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0442      0.166      6.287      0.000       0.719       1.370
ar.L1          1.1108      0.025     43.748      0.000       1.061       1.161
ar.L2         -0.2070      0.023     -9.011      0.0

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35056
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -83226.701
Date:                Mon, 08 May 2023   AIC                         166463.403
Time:                        11:55:12   BIC                         166505.726
Sample:                    01-01-2020   HQIC                        166476.883
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0461      0.166      6.300      0.000       0.721       1.372
ar.L1          1.1107      0.025     43.762      0.000       1.061       1.160
ar.L2         -0.2069      0.023     -9.009      0.0

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35076
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -83273.793
Date:                Mon, 08 May 2023   AIC                         166557.585
Time:                        11:55:15   BIC                         166599.911
Sample:                    01-01-2020   HQIC                        166571.066
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0448      0.166      6.297      0.000       0.720       1.370
ar.L1          1.1131      0.025     44.087      0.000       1.064       1.163
ar.L2         -0.2091      0.023     -9.154      0.0

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35096
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -83311.719
Date:                Mon, 08 May 2023   AIC                         166633.437
Time:                        11:55:18   BIC                         166675.766
Sample:                    01-01-2020   HQIC                        166646.918
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0462      0.166      6.308      0.000       0.721       1.371
ar.L1          1.1131      0.025     44.109      0.000       1.064       1.163
ar.L2         -0.2091      0.023     -9.159      0.0

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35116
Model:                 ARIMA(2, 0, 1)   Log Likelihood              -83354.632
Date:                Mon, 08 May 2023   AIC                         166719.264
Time:                        11:55:25   BIC                         166761.596
Sample:                    01-01-2020   HQIC                        166732.746
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0471      0.166      6.321      0.000       0.722       1.372
ar.L1          1.1312      0.025     45.019      0.000       1.082       1.180
ar.L2         -0.2253      0.023     -9.913      0.0

In [6]:
mse2 = []

for train_indx, val_indx in tss.split(df):
    train = df.iloc[train_indx]
    test = df.iloc[val_indx]
    
    mod = ARIMA(train['310_PV_1'], order = (8,1,1))
    result = mod.fit()
    print(result.summary())
    
    start = len(train)
    end = len(train)+1
    
    pred = result.predict(start = start, end = end)
    
    
    error = 0.5*(test['310_PV_1'][0] - pred[0])**2
    mse2.append(error)
    
    #print(pred)
    
    #print(test['309_WIND_1'][0])
    #print(pred[0])
print(mse2)
    

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35036
Model:                 ARIMA(8, 1, 1)   Log Likelihood              -83182.255
Date:                Mon, 08 May 2023   AIC                         166384.510
Time:                        11:55:49   BIC                         166469.151
Sample:                    01-01-2020   HQIC                        166411.469
                         - 12-30-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9767      0.002    448.124      0.000       0.972       0.981
ar.L2         -0.0770      0.003    -28.421      0.000      -0.082      -0.072
ar.L3         -0.0027      0.003     -0.932      0.3

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35056
Model:                 ARIMA(8, 1, 1)   Log Likelihood              -83219.991
Date:                Mon, 08 May 2023   AIC                         166459.982
Time:                        11:56:14   BIC                         166544.629
Sample:                    01-01-2020   HQIC                        166486.942
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9767      0.002    448.436      0.000       0.972       0.981
ar.L2         -0.0770      0.003    -28.436      0.000      -0.082      -0.072
ar.L3         -0.0027      0.003     -0.932      0.3

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35076
Model:                 ARIMA(8, 1, 1)   Log Likelihood              -83267.056
Date:                Mon, 08 May 2023   AIC                         166554.112
Time:                        11:56:40   BIC                         166638.765
Sample:                    01-01-2020   HQIC                        166581.073
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9771      0.002    448.634      0.000       0.973       0.981
ar.L2         -0.0772      0.003    -28.514      0.000      -0.082      -0.072
ar.L3         -0.0028      0.003     -0.975      0.3

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35096
Model:                 ARIMA(8, 1, 1)   Log Likelihood              -83305.024
Date:                Mon, 08 May 2023   AIC                         166630.048
Time:                        11:57:08   BIC                         166714.707
Sample:                    01-01-2020   HQIC                        166657.011
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9771      0.002    448.886      0.000       0.973       0.981
ar.L2         -0.0772      0.003    -28.523      0.000      -0.082      -0.072
ar.L3         -0.0028      0.003     -0.981      0.3

C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)
C:\Users\bvits\anaconda3\envs\modern_ts\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency 15T will be used.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:               310_PV_1   No. Observations:                35116
Model:                 ARIMA(8, 1, 1)   Log Likelihood              -83347.951
Date:                Mon, 08 May 2023   AIC                         166715.903
Time:                        11:57:33   BIC                         166800.567
Sample:                    01-01-2020   HQIC                        166742.866
                         - 12-31-2020                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9773      0.002    449.103      0.000       0.973       0.982
ar.L2         -0.0773      0.003    -28.580      0.000      -0.083      -0.072
ar.L3         -0.0028      0.003     -0.992      0.3

In [8]:
mse3 = []

for train_indx, val_indx in tss.split(df):
    train = df.iloc[train_indx]
    test = df.iloc[val_indx]
    
    pred = train['310_PV_1'][-1]
    
    
    error = 0.5*(test['310_PV_1'][0] - pred)**2
    mse3.append(error)
print(mse3)


[9.66227718579457e-08, 9.637964621242322e-08, 0.06275144245009717, 4.1972275624956465e-06, 4.852593834614524e-07]
